In [ ]:
%matplotlib inline

In [ ]:
from pathlib import Path

from spectral_cube import SpectralCube
# from astropy.stats import mad_std

In [ ]:
# !ls /reduction10/erickoch/LGLBS/line_imaging/derived/ic10/


In [ ]:
%run c+d_analysis/hi_galaxy_params.py

In [ ]:
data_path = Path("/reduction/erickoch/LGLBS/line_imaging/derived/")


ic10_cube = SpectralCube.read(data_path / "ic10/ic10_C+D+tp_hi21cm_0p8kms_120pc.fits")
ngc6822_cube = SpectralCube.read(data_path / "ngc6822/ngc6822_C+D+tp_hi21cm_0p8kms_120pc.fits")
ic1613_cube = SpectralCube.read(data_path / "ic1613/ic1613_C+D+tp_hi21cm_0p8kms_120pc.fits")
wlm_cube = SpectralCube.read(data_path / "wlm/wlm_C+D+tp_hi21cm_0p8kms_120pc.fits")

m33_cube = SpectralCube.read(data_path / "m33/m33_C+D+tp_himidres_120pc.fits")

m31_cube = SpectralCube.read(data_path / "m31/m31_C+D+tp_hilores_120pc.fits")



In [ ]:
ic10_cube.beam

In [ ]:
ngc6822_cube.beam

In [ ]:
ic1613_cube.beam

In [ ]:
wlm_cube.beam

In [ ]:
m33_cube.beam

In [ ]:
m31_cube.beam

In [ ]:
wlm_cube.header['CDELT2'] * 600 * 200

In [ ]:
ic10_dist = gal_ic10.distance
ngc6822_dist = gal_ngc6822.distance
ic1613_dist = gal_ic1613.distance
wlm_dist = gal_wlm.distance
m33_dist = gal_m33.distance
m31_dist = gal_m31.distance


In [ ]:
def center_slice(spat_shape, width=100):
    centy = spat_shape[0] // 2
    centx = spat_shape[1] // 2

    return (slice(centy-width, centy+width),
            slice(centy-width, centy+width))


def center_mad_std(cube):

    rms_noise = cube[-10:][(slice(None),) + center_slice(cube[0].shape)].mad_std()
    return rms_noise


In [ ]:


ic10_cube[-10:].mad_std(0).quicklook()
plt.colorbar()

In [ ]:
ngc6822_cube[-10:].mad_std(0).quicklook()
plt.colorbar()

In [ ]:
ic1613_cube[-10:].mad_std(0).quicklook()
plt.colorbar()

In [ ]:
wlm_cube[-10:].mad_std(0).quicklook()
plt.colorbar()

In [ ]:
m33_cube[-10:].mad_std(0).quicklook()
plt.colorbar()

In [ ]:
m31_cube[-10:].mad_std(0).quicklook()
plt.colorbar()

In [ ]:
ic10_rms = center_mad_std(ic10_cube)

ic10_rms

In [ ]:
ngc6822_rms = center_mad_std(ngc6822_cube)

ngc6822_rms

In [ ]:
ic1613_rms = center_mad_std(ic1613_cube)

ic1613_rms

In [ ]:
wlm_rms = center_mad_std(wlm_cube)

wlm_rms

In [ ]:
m33_rms = center_mad_std(m33_cube)

m33_rms

In [ ]:
m31_rms = center_mad_std(m31_cube)

m31_rms

In [ ]:
# chan_width = np.abs(np.diff(ngc6822_cube.spectral_axis)[0]).to(u.km / u.s)

# chan_width

target_width = 10 * u.km / u.s

def rms_coldens(rms, chan_width, target_width):

    rms_at_target = rms / np.sqrt(target_width / chan_width).to(u.one)
    rms_coldens = 1.82e18 * (u.cm**-2 / (u.K * u.km / u.s)) * rms_at_target * target_width.to(u.km / u.s)

    print(rms_at_target, rms_coldens)

    return rms_coldens

def rms_surfdens(rms, chan_width, target_width):

    rms_at_target = rms / np.sqrt(target_width / chan_width).to(u.one)
    rms_surfdens = (0.0196 / 1.4) * (u.solMass * u.pc**-2 / (u.K * u.km / u.s)) * rms_at_target * target_width.to(u.km / u.s)

    print(rms_at_target, rms_surfdens)

    return rms_surfdens

def beam_phys_size(beam, distance):
    return beam.sr.to(u.rad**2).value * distance.to(u.pc)**2

def rms_himass(coldens, beam, distance):

    beam_area = beam_phys_size(beam, distance)

    print(beam, beam_area)

    hi_mass = c.m_p.to(u.solMass) * (coldens * beam_area).to(u.one)

    print(hi_mass)

    return hi_mass

In [ ]:
chan_width = np.abs(np.diff(ic10_cube.spectral_axis)[0]).to(u.km / u.s)


ic10_rms_coldens = rms_coldens(ic10_rms, chan_width, target_width)
ic10_rms_surfdens = rms_surfdens(ic10_rms, chan_width, target_width)

ic10_rms_himass = rms_himass(ic10_rms_coldens, ic10_cube.beam, ic10_dist)


In [ ]:
chan_width = np.abs(np.diff(ngc6822_cube.spectral_axis)[0]).to(u.km / u.s)

ngc6822_rms_coldens = rms_coldens(ngc6822_rms, chan_width, target_width)
ngc6822_rms_surfdens = rms_surfdens(ngc6822_rms, chan_width, target_width)


ngc6822_rms_himass = rms_himass(ngc6822_rms_coldens, ngc6822_cube.beam, ngc6822_dist)


In [ ]:
chan_width = np.abs(np.diff(ic1613_cube.spectral_axis)[0]).to(u.km / u.s)

ic1613_rms_coldens = rms_coldens(ic1613_rms, chan_width, target_width)
ic1613_rms_surfdens = rms_surfdens(ic1613_rms, chan_width, target_width)

ic1613_rms_himass = rms_himass(ic1613_rms_coldens, ic1613_cube.beam, ic1613_dist)


In [ ]:
chan_width = np.abs(np.diff(wlm_cube.spectral_axis)[0]).to(u.km / u.s)

wlm_rms_coldens = rms_coldens(wlm_rms, chan_width, target_width)
wlm_rms_surfdens = rms_surfdens(wlm_rms, chan_width, target_width)

wlm_rms_himass = rms_himass(wlm_rms_coldens, wlm_cube.beam, wlm_dist)


In [ ]:
chan_width = np.abs(np.diff(m33_cube.spectral_axis)[0]).to(u.km / u.s)

m33_rms_coldens = rms_coldens(m33_rms, chan_width, target_width)
m33_rms_surfdens = rms_surfdens(m33_rms, chan_width, target_width)

m33_rms_himass = rms_himass(m33_rms_coldens, m33_cube.beam, m33_dist)


In [ ]:
chan_width = np.abs(np.diff(m31_cube.spectral_axis)[0]).to(u.km / u.s)

m31_rms_coldens = rms_coldens(m31_rms, chan_width, target_width)
m31_rms_surfdens = rms_surfdens(m31_rms, chan_width, target_width)

m31_rms_himass = rms_himass(m31_rms_coldens, m31_cube.beam, m31_dist)


In [ ]:
print(ic10_rms_coldens * 5)
print(ic1613_rms_coldens * 5)
print(ngc6822_rms_coldens * 5)
print(wlm_rms_coldens * 5)
print(m33_rms_coldens * 5)
print(m31_rms_coldens * 5)


In [ ]:
print(ic10_rms_surfdens * 5)
print(ic1613_rms_surfdens * 5)
print(ngc6822_rms_surfdens * 5)
print(wlm_rms_surfdens * 5)
print(m33_rms_surfdens * 5)
print(m31_rms_surfdens * 5)


In [ ]:
print(ic10_rms_himass * 5)
print(ic1613_rms_himass * 5)
print(ngc6822_rms_himass * 5)
print(wlm_rms_himass * 5)
print(m33_rms_himass * 5)
print(m31_rms_himass * 5)
